In [2]:
import os 

In [3]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
load_dotenv(override=True) 
api_key = os.getenv('OPENAI_API_KEY')

In [5]:
MODEL = 'gpt-4o-mini'

openai = OpenAI()

In [6]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


class Website: 
    def __init__(self,url):
        self.url = url 
        response = requests.get(url, headers = headers) 
        self.body = response.content 
        soup = BeautifulSoup(self.body,'html.parser') 
        self.title = soup.title.string if soup.title else "No title found"

        if soup.body: 
            for irrelevant in soup.body(['script','style','img','input']): 
                irrelevant.decompose() 
            self.text = soup.body.get_text(separator = "\n", strip = True)
        else: 
            self.text = ""
        
        links = [link.get('href') for link in soup.find_all('a')] 

        self.links = [link for link in links if link] 
    
    def get_contents(self): 
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n" 





In [7]:
ed = Website('https://edwarddonner.com') 
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of pr

In [8]:
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

In [9]:
link_system_prompt= """ You are provided with a list of links found on a webpage \n 
You are able to decide which of these links would be most relevant to include in a brochure about a company \n 
such as links to an about page, or company page or careers/jobs pages \n 
"""

link_system_prompt += """You should respond in JSON as in this example:""" 
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""


In [10]:
link_system_prompt

' You are provided with a list of links found on a webpage \n \nYou are able to decide which of these links would be most relevant to include in a brochure about a company \n \nsuch as links to an about page, or company page or careers/jobs pages \n \nYou should respond in JSON as in this example:\n{\n    "links": [\n        {"type": "about page", "url": "https://full.url/goes/here/about"},\n        {"type": "careers page": "url": "https://another.full.url/careers"}\n    ]\n}\n'

In [11]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} -"
    user_prompt += "please decide which of these links are relevant web links for a brochure about the company, respond with full https URL\n"
    user_prompt += "Do not include Terms of Service, Privacy, email links. \n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)

    return user_prompt

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)},
        ],
        response_format={"type": "json_object"},
    )

    result = response.choices[0].message.content

    return json.loads(result)

In [13]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")

In [14]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}

In [18]:
hf = Website("https://huggingface.co") 

result = "Landing page:\n" 
result += hf.get_contents() 

links = get_links("https://huggingface.co")

links

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [21]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)

    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()

    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [23]:
system_prompt = """You are a assistant that analyzes the content of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown 
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [35]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [36]:
def get_brochure_user_prompt(company_name, url): 
    user_prompt = f"You are looking at a company called {company_name}\n" 
    user_prompt += f"Here are contents of its landing page and other relevant pages; use this information to build a short brochure of company in markdown. \n"
    user_prompt += get_all_details(url) 
    user_prompt = user_prompt[:20_000] 
    return user_prompt

In [38]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
    )

    result = response.choices[0].message.content 

    display(Markdown(result))

In [39]:
# create_brochure('huggingface','https://huggingface.co')

In [40]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
        stream=True,
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True) 

    for chunk in stream: 
        response += chunk.choices[0].delta.content or '' 
        response = response.replace("```","").replace('markdown','') 
        update_display(Markdown(response), display_id=display_handle.display_id)

In [41]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Welcome to Hugging Face! 🤗
*Where AI meets community, collaboration, and maybe a bit of caffeine!*

---

## Who are we?
At **Hugging Face**, we believe in one thing: making AI as cuddly as a teddy bear! We're not just another machine learning company; we're the friendly neighborhood superheroes of the AI world, collaborating and building the future together. Picture a place where models and datasets throw epic parties, and everyone is invited!

---

## Our Vibe 🌟
- **Join the AI Revolution**: Over **50,000 organizations** (yep, even your distant cousin's startup) are already grooving with us. From tech giants like **Meta** and **Google** to innovative companies like **Grammarly**, we’re all about building connections!
  
- **Fun-tastic Culture**: Have a laugh, share some memes, and let creativity flow. We promote open-source cooperation, and everyone gets a say! No superhero without sidekicks, right?

- **The Future is Bright**: Run your own models, generate datasets, and dive deep into application spaces. Who knew AI could be this adventurous?

---

## What We Offer 👾
- **Models Galore**: Browse **400k+ models**. From chatbots that understand your dad jokes to 3D text generators that can make anything, we've got it all! 

- **Dazzling Datasets**: Turn data into wisdom with **100k+ datasets**! Whether it's NLP, computer vision, or just high scores in video games, we've got your back.

- **Spaces to Shine**: Share your AI applications, make new friends, and let the community rave about your innovations! Spaces are like smaller universes where creativity reigns.

---

## Want in on the Action? 🚪
### **Careers at Hugging Face**
Join our quest for **AI utopia**! We’re hiring talented folks who want to **democratize machine learning** and have some fun while doing it. 

Roles include:
- **ML Engineers** - Become the wizardry behind the models! 
- **Data Scientists** - Turn numbers into gold!
- **Community Managers** - Keep the vibe alive and thriving!

*P.S.: Your pet can be your interview buddy. We're serious!* 

---

## Pricing Made Simple 💰
- **HF Hub**: Collaborate for FREE! All public models and datasets at your fingertips.
- **Pro Account**: For just $9/month, unleash supercharged features and get a shiny badge just for you!
- **Enterprise Hub**: Get enterprise-grade security and support starting at just $20/user/month. Because superheroes need protection too!

---

## Come Hug It Out! 🤗
- Check out our blog for amusing articles, hilarious discussions, and updates on what's hot in AI. Spoiler: it's always *the new model*!

**Visit us now**: [Hugging Face](https://huggingface.co) and become part of a community that’s as warm and fuzzy as a hug!

*Remember, in a world where you can choose to be anything, be a Hugging Face!*


In [ ]:
 - zero shot 
- translation 
